In [ ]:
import pandas as pd
import pickle

In [ ]:
# Extract ml info
df_extra_features_train_raw = pd.read_csv('../../Data/ml_train.csv')
df_extra_features_test_raw = pd.read_csv('../../Data/ml_test.csv')

In [ ]:
df_extra_features_test_raw = pd.read_csv('../../Data/ml_test.csv')# select those columns that Angel used to fit the ML models
selected_columns = ['study_id','site_id','bio01','bio02','bio05','bio08','bio14','ec','elevation','es','gHM','pdsi','soil_den_b10','moss','shrub','management','x0_1.0','x0_2.0','x0_4.0','x0_5.0','x0_7.0','x0_8.0','x0_10.0','x0_12.0']

df_extra_features_train = df_extra_features_train_raw[selected_columns]
df_extra_features_test = df_extra_features_test_raw[selected_columns]

# Note: it seems that these partitions are different from those used to calibrate DI-ML
# df_train.shape : (790, 25)
# df_test.shape : (217, 25)

df_ML_extra_features_info = pd.concat([df_extra_features_train, df_extra_features_test], ignore_index=True)

In [ ]:
df_train_global_regressor = pd.read_csv('../../Data/data_train_global_regressor_V0.csv')
df_test_global_regressor = pd.read_csv('../../Data/data_test_global_regressor_V0.csv')

In [ ]:
df_ML_train = df_train_global_regressor.merge(df_ML_extra_features_info, on=['study_id','site_id'], how='left')
df_ML_test = df_test_global_regressor.merge(df_ML_extra_features_info, on=['study_id','site_id'], how='left')

In [ ]:
df_ML_train.to_csv('../../Data/df_ML_train_global_regressor_V0.csv', index=False)
df_ML_test.to_csv('../../Data/df_ML_test_global_regressor_V0.csv', index=False)

In [ ]:
# sanity check
rows_with_na = df_ML_train.isna().any(axis=1)
number_rows_with_na = rows_with_na.sum()
print(number_rows_with_na) # 27 in the training set

# sanity check
rows_with_na = df_ML_test.isna().any(axis=1)
number_rows_with_na = rows_with_na.sum()
print(number_rows_with_na) # 20 in the test set

In [ ]:
# Drop NAs from dataframes
df_ML_train_without_na = df_ML_train.dropna()
df_ML_test_without_na = df_ML_test.dropna()

In [ ]:
# Create an iterator for cross validation
df_folds = pd.read_csv('../../Data/CV_folds.csv')
df_ML_train_studyid_series = df_ML_train_without_na['study_id']
df_ML_train_studyid = df_ML_train_studyid_series.to_frame(name='study_id')
df_ML_train_folds = df_ML_train_studyid.merge(df_folds, on=['study_id'], how='left')

myCViterator = []
for i in range(0,5):
    trainIndices = df_ML_train_folds[df_ML_train_folds['fold'] != i].index.values.astype(int)
    testIndices = df_ML_train_folds[df_ML_train_folds['fold'] == i].index.values.astype(int)
    myCViterator.append((trainIndices, testIndices))

# Save iterator    
with open('../../Data/myCViterator_PREVENT.pkl', 'wb') as f:
    pickle.dump(myCViterator, f)


In [ ]:
# Import our cv iterator (K-Folds)
with open('../../Data/myCViterator_PREVENT.pkl', 'rb') as file:
        myCViterator = pickle.load(file)

In [ ]:
import numpy as np
from sklearn.linear_model import BayesianRidge
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import spearmanr, uniform
from sklearn.preprocessing import StandardScaler

# Define columns for CNN and non-CNN features
X_columns_CNN = ['global_regressor_V0','bio01','bio02','bio05','bio08','bio14','ec','elevation','es','gHM','pdsi','soil_den_b10','moss','shrub','management_y','x0_1.0','x0_2.0','x0_4.0','x0_5.0','x0_7.0','x0_8.0','x0_10.0','x0_12.0']
X_columns_NO_CNN = ['bio01','bio02','bio05','bio08','bio14','ec','elevation','es','gHM','pdsi','soil_den_b10','moss','shrub','management_y','x0_1.0','x0_2.0','x0_4.0','x0_5.0','x0_7.0','x0_8.0','x0_10.0','x0_12.0']

y_columns = ['log_vr_total']

# Select training and test sets
X_train_CNN = df_ML_train_without_na[X_columns_CNN]
X_train_NO_CNN = df_ML_train_without_na[X_columns_NO_CNN]
X_test_CNN = df_ML_test_without_na[X_columns_CNN]
X_test_NO_CNN = df_ML_test_without_na[X_columns_NO_CNN]

# Select columns by name
columns_to_scale_CNN = ['global_regressor_V0','bio01','bio02','bio05','bio08','bio14','ec','elevation','es','gHM','pdsi','soil_den_b10','moss','shrub']
columns_to_scale_NO_CNN = ['bio01','bio02','bio05','bio08','bio14','ec','elevation','es','gHM','pdsi','soil_den_b10','moss','shrub']

# Create an instance of StandardScaler
scaler_CNN = StandardScaler()
scaler_NO_CNN = StandardScaler()

# Fit the scaler only with the training data and then transform the training data
X_train_CNN[columns_to_scale_CNN] = scaler_CNN.fit_transform(X_train_CNN[columns_to_scale_CNN])
X_train_NO_CNN[columns_to_scale_NO_CNN] = scaler_NO_CNN.fit_transform(X_train_NO_CNN[columns_to_scale_NO_CNN])

# Apply the transformation to the test data using the fitted values from the training data
X_test_CNN[columns_to_scale_CNN] = scaler_CNN.transform(X_test_CNN[columns_to_scale_CNN])
X_test_NO_CNN[columns_to_scale_NO_CNN] = scaler_NO_CNN.transform(X_test_NO_CNN[columns_to_scale_NO_CNN])

# Select target variables
y_train = df_ML_train_without_na[y_columns]
y_test = df_ML_test_without_na[y_columns]

In [ ]:
# Model definition
model_bayesian_ridge = BayesianRidge(n_iter=25000)

# Define the hyperparameter grid to explore
param_grid = {
    'alpha_1': uniform(loc=0, scale=1),
    'alpha_2': uniform(loc=0, scale=1),
    'lambda_1': uniform(loc=0, scale=1),
    'lambda_2': uniform(loc=0, scale=1),
    'fit_intercept': [False, True]
}

# Cross validation strategy (here we use K-Fold)
cv = KFold(n_splits = 5, shuffle = True, random_state = 42)

# Random search of best hyperparameter combinations
grid_search = RandomizedSearchCV(model_bayesian_ridge, param_distributions = param_grid, cv = myCViterator,
                            scoring = 'neg_mean_absolute_error', n_iter = 10000,
                            verbose = 2, random_state = 135, n_jobs = 8)

grid_search.fit(X_train_CNN, y_train)

# best hyperparameter combination
best_hyperparameters_CNN = grid_search.best_params_

# best model
best_model_CNN = grid_search.best_estimator_


In [ ]:
best_model_CNN

In [ ]:
# make predictions
y_pred_CNN = best_model_CNN.predict(X_test_CNN)

# Evaluation of our best model
mae = mean_absolute_error(y_test, y_pred_CNN)
mse = mean_squared_error(y_test, y_pred_CNN)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_CNN)

# Estimate spearman rho
coef_spearman, p_valor_spearman = spearmanr(y_test, y_pred_CNN)

# Best model results
# print("Model coefficients:", best_model_CNN.coef_)
# print("Intercept:", best_model_CNN.intercept_)
print("Mean square error (MSE):", mse)
print("Root mean square error (RMSE):", rmse)
print("Mean absolute error (MAE):", mae)
print("R-squared:", r2)
print("Spearman-rho:", coef_spearman,"(pvalor:",p_valor_spearman,")")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

y_true = y_test.to_numpy()
y_true_array = y_true.flatten()
# fit a regression line with numpy
coef = np.polyfit(y_true_array, y_pred_CNN, 1)
poly1d_fn = np.poly1d(coef)

# create a scatter plot with regression line
plt.scatter(y_true_array, y_pred_CNN, label='Data')
plt.plot(y_true_array, poly1d_fn(y_true_array), color = 'red', label = 'Linear regression')
plt.plot(sorted(y_true_array), sorted(y_true_array), color = 'blue', linestyle = '--', label = 'Expected trend')

# Add labels
plt.xlabel('Observed visitation rate')
plt.ylabel('Predicted visitation rate')
plt.title('Results for BayRidge regressor with the CNN results at global scale')

# show legend
plt.legend()

# show plot
plt.show()

In [ ]:
# Get coef. from the best model estimated
coeficients = best_model_CNN.coef_

# Get variance from the estimated coeficients
variance_coefs = np.var(coeficients)

# Estimate VIP for each variable
VIP = np.abs(coeficients) / variance_coefs

# Sort VIP
sorted_indices = np.argsort(VIP)[::-1]  # Sort from larger to smaller VIP
sorted_VIP = VIP[sorted_indices]

# Print VIP
for i, idx in enumerate(sorted_indices):
    print(f"{X_columns_CNN[idx]}: VIP = {sorted_VIP[i]}")
    
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_indices)), sorted_VIP, align='center')
plt.yticks(range(len(sorted_indices)), [X_columns_CNN[i] for i in sorted_indices])
plt.xlabel('VIP (Variable Importance Plot)')
plt.ylabel('Variable')
plt.title('Variable Importance Plot for the Bayesian Ridge Regression model with CNN info')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# and using grandient boost??

In [ ]:
# What if we eliminate highly correlated variables before fitting the model?

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(df, columns):
    vif_data = pd.DataFrame()
    vif_data["feature"] = columns
    vif_data["VIF"] = [variance_inflation_factor(df[columns].values, i) for i in range(len(columns))]
    return vif_data

X_columns_CNN_raw = ['global_regressor_V0','bio01','bio02','bio05','bio08','bio14','ec','elevation','es','gHM','pdsi','soil_den_b10','moss','shrub','management_y','x0_1.0','x0_2.0','x0_4.0','x0_5.0','x0_7.0','x0_8.0','x0_10.0','x0_12.0']
X_columns_NO_CNN_raw = ['bio01','bio02','bio05','bio08','bio14','ec','elevation','es','gHM','pdsi','soil_den_b10','moss','shrub','management_y','x0_1.0','x0_2.0','x0_4.0','x0_5.0','x0_7.0','x0_8.0','x0_10.0','x0_12.0']

y_columns = ['log_vr_total']

X_train_CNN_raw = df_ML_train_without_na[X_columns_CNN_raw]
X_train_NO_CNN_raw = df_ML_train_without_na[X_columns_NO_CNN_raw]
X_test_CNN_raw = df_ML_test_without_na[X_columns_CNN_raw]
X_test_NO_CNN_raw = df_ML_test_without_na[X_columns_NO_CNN_raw]


# Estimate VIF for X_train_CNN
vif_CNN = calculate_vif(X_train_CNN_raw, X_columns_CNN_raw)
print("VIF for X_train_CNN:")
print(vif_CNN)

# Estimate VIF for X_train_NO_CNN
vif_NO_CNN = calculate_vif(X_train_NO_CNN_raw, X_columns_NO_CNN_raw)
print("VIF for X_train_NO_CNN:")
print(vif_NO_CNN)

In [ ]:
# Define a threshold for VIF
vif_threshold = 8

# Select columns that have VIF less than the threshold
selected_columns_CNN_raw = vif_CNN[vif_CNN["VIF"] < vif_threshold]["feature"].tolist()
selected_columns_NO_CNN_raw = vif_NO_CNN[vif_NO_CNN["VIF"] < vif_threshold]["feature"].tolist()

# Create new dataframes with the selected columns
X_train_CNN_raw_reduced = X_train_CNN_raw[selected_columns_CNN_raw]
X_train_NO_CNN_raw_reduced = X_train_NO_CNN_raw[selected_columns_NO_CNN_raw]
X_test_CNN_raw_reduced = X_test_CNN_raw[selected_columns_CNN_raw]
X_test_NO_CNN_raw_reduced = X_test_NO_CNN_raw[selected_columns_NO_CNN_raw]

print("Columns selected for X_train_CNN:", selected_columns_CNN_raw)
print("Columns selected for X_train_NO_CNN:", selected_columns_NO_CNN_raw)

In [ ]:
# Scale columns selected 
scaler_CNN_raw = StandardScaler()
scaler_NO_CNN_raw = StandardScaler()

# Fit the scaler with just the training data and then transform the training data
X_train_CNN_raw_reduced_sc = scaler_CNN_raw.fit_transform(X_train_CNN_raw_reduced)
X_train_NO_CNN_raw_reduced_sc = scaler_NO_CNN_raw.fit_transform(X_train_NO_CNN_raw_reduced)

In [ ]:
grid_search.fit(X_train_CNN_raw_reduced_sc, y_train)

# best hyperparameter combination
best_hyperparameters_CNN = grid_search.best_params_

# best model
best_model_CNN_reduced = grid_search.best_estimator_

In [ ]:
# make predictions
y_pred_CNN_raw_reduced = best_model_CNN_reduced.predict(X_test_CNN_raw_reduced)

# Evaluation of our best new model
mae_raw_reduced = mean_absolute_error(y_test, y_pred_CNN_raw_reduced)
mse_raw_reduced = mean_squared_error(y_test, y_pred_CNN_raw_reduced)
rmse_raw_reduced = np.sqrt(mse_raw_reduced)
r2_raw_reduced = r2_score(y_test, y_pred_CNN_raw_reduced)

# Estimate spearman rho
coef_spearman_raw_reduced, p_valor_spearman_raw_reduced = spearmanr(y_test, y_pred_CNN_raw_reduced)

# Best model results
# print("Model coefficients:", best_model_CNN.coef_)
# print("Intercept:", best_model_CNN.intercept_)
print("Mean square error (MSE):", mse_raw_reduced)
print("Root mean square error (RMSE):", rmse_raw_reduced)
print("Mean absolute error (MAE):", mae_raw_reduced)
print("R-squared:", r2_raw_reduced)
print("Spearman-rho:", coef_spearman_raw_reduced,"(pvalor:",p_valor_spearman_raw_reduced,")")

In [ ]:
# Is the CNN prediction introducing noise??? it's possible

In [ ]:
# Fit model without CNN info (global regressor)
grid_search.fit(X_train_NO_CNN, y_train)

# best hyperparameter combination
best_hyperparameters_NO_CNN = grid_search.best_params_

# best model
best_model_NO_CNN = grid_search.best_estimator_


In [ ]:
best_model_NO_CNN

In [ ]:
# make predictions
y_pred_NO_CNN = best_model_NO_CNN.predict(X_test_NO_CNN)

# Evaluation of our best model
mae_NO_CNN = mean_absolute_error(y_test, y_pred_NO_CNN)
mse_NO_CNN = mean_squared_error(y_test, y_pred_NO_CNN)
rmse_NO_CNN = np.sqrt(mse_NO_CNN)
r2_NO_CNN = r2_score(y_test, y_pred_NO_CNN)

# Estimate spearman rho
coef_spearman_NO_CNN, p_valor_spearman_NO_CNN = spearmanr(y_test, y_pred_NO_CNN)

# Best model results
# print("Model coefficients:", best_model_NO_CNN.coef_)
# print("Intercept:", best_model_NO_CNN.intercept_)
print("Mean square error (MSE):", mse_NO_CNN)
print("Root mean square error (RMSE):", rmse_NO_CNN)
print("R-squared:", r2_NO_CNN)
print("Mean absolute error (MAE):", mae_NO_CNN)
print("Spearman-rho:", coef_spearman_NO_CNN,"(pvalor:",p_valor_spearman_NO_CNN,")")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

y_true = y_test.to_numpy()
y_true_array = y_true.flatten()
# fit a regression line with numpy
coef = np.polyfit(y_true_array, y_pred_NO_CNN, 1)
poly1d_fn = np.poly1d(coef)

# create a scatter plot with regression line
plt.scatter(y_true_array, y_pred_NO_CNN, label='Data')
plt.plot(y_true_array, poly1d_fn(y_true_array), color = 'red', label = 'Linear regression')
plt.plot(sorted(y_true_array), sorted(y_true_array), color = 'blue', linestyle = '--', label = 'Expected trend')

# Add labels
plt.xlabel('Observed visitation rate')
plt.ylabel('Predicted visitation rate')
plt.title('Results for BayRidge regressor without CNN info at global scale')

# show legend
plt.legend()

# show plot
plt.show()

In [ ]:
# Get coef. from the best model estimated
coeficients_NO_CNN = best_model_NO_CNN.coef_

# Get variance from the estimated coeficients
variance_coefs_NO_CNN = np.var(coeficients_NO_CNN)

# Estimate VIP for each variable
VIP_NO_CNN = np.abs(coeficients_NO_CNN) / variance_coefs_NO_CNN

# Sort VIP
sorted_indices_NO_CNN = np.argsort(VIP_NO_CNN)[::-1]  # Sort from larger to smaller VIP
sorted_VIP_NO_CNN = VIP_NO_CNN[sorted_indices_NO_CNN]

# Print VIP
for i, idx in enumerate(sorted_indices_NO_CNN):
    print(f"{X_columns_NO_CNN[idx]}: VIP = {sorted_VIP_NO_CNN[i]}")
    
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_indices_NO_CNN)), sorted_VIP_NO_CNN, align='center')
plt.yticks(range(len(sorted_indices_NO_CNN)), [X_columns_NO_CNN[i] for i in sorted_indices_NO_CNN])
plt.xlabel('VIP (Variable Importance Plot)')
plt.ylabel('Variable')
plt.title('Variable Importance Plot for the Bayesian Ridge Regression model WITHOUT CNN info')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
y_pred_test_NO_CNN = best_model_NO_CNN.predict(X_test_NO_CNN)
df_ML_test_without_na['BayReg'] = y_pred_test_NO_CNN.flatten()

y_pred_train_NO_CNN = best_model_NO_CNN.predict(X_train_NO_CNN)
df_ML_train_without_na['BayReg'] = y_pred_train_NO_CNN.flatten()

In [ ]:
# Let's adjust by adding the BayReg data and the rest of the Bayreg+CNN variables

from sklearn.preprocessing import StandardScaler
import numpy as np

# Add BayReg predictions as a new column
X_train_CNN_Bay_others = np.hstack((X_train_CNN, y_pred_train_NO_CNN.reshape(-1, 1)))
X_test_CNN_Bay_others = np.hstack((X_test_CNN, y_pred_test_NO_CNN.reshape(-1, 1)))

# Scale features
scaler_CNN_Bay_others = StandardScaler()

# Adjust and transform the training set
X_train_CNN_Bay_others = scaler_CNN_Bay_others.fit_transform(X_train_CNN_Bay_others)

# Adjust and transform the test set
X_test_CNN_Bay_others = scaler_CNN_Bay_others.transform(X_test_CNN_Bay_others)

In [ ]:
# We try to do an optimized search for parameters for the neural regressor

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr
from torch.utils.data import DataLoader, TensorDataset
from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV

# Define a neural network model with batch normalization and dropout
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim1=64, hidden_dim2=32, dropout_rate=0.5):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.bn1 = nn.BatchNorm1d(hidden_dim1)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.bn2 = nn.BatchNorm1d(hidden_dim2)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(hidden_dim2, 1)
    
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Create a NeuralNetRegressor
net = NeuralNetRegressor(
    NeuralNetwork,
    module__input_dim=X_train_CNN_Bay_others.shape[1],
    max_epochs=50,
    lr=0.01,
    optimizer=optim.Adam,
    iterator_train__shuffle=True,
    device='cuda' if torch.cuda.is_available() else 'cpu',
)

# Define the hyperparameter grid
params = {
    'lr': [0.1],
    'max_epochs': [50],
    'module__hidden_dim1': [32],
    'module__hidden_dim2': [32],
    'module__dropout_rate': [.679,.68,.685,.689,.69,.691],
}

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_CNN_Bay_others, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_CNN_Bay_others, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Perform hyperparameter search
gs = GridSearchCV(net, params, refit=True, cv=myCViterator, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
gs.fit(X_train_tensor, y_train_tensor)

print("Best hyperparameters found:")
print(gs.best_params_)

# Evaluate the model on the test set
y_pred_test = gs.predict(X_test_tensor)

# Calculate the Spearman coefficient
rho = spearmanr(y_test, y_pred_test.flatten())[0]
print(f"Spearman's rho: {rho:.4f}")

In [ ]:
# Function to find NN model with a higher Spearman rho value than the BayRidge model

In [ ]:
import random

# Set the seed for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# Set the seed
seed = 42
set_seed(seed)

def find_best_model(X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, net, params, myCViterator, rho_threshold):
    rho = 0  # Initialize rho
    best_params = None

    while rho <= rho_threshold:
        # Search hyperparameters
        gs = GridSearchCV(net, params, refit=True, cv=myCViterator, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
        gs.fit(X_train_tensor, y_train_tensor)

        # Eval model
        y_pred_test = gs.predict(X_test_tensor)

        # Estimate its Spearman's rho
        rho = spearmanr(y_test_tensor, y_pred_test.flatten())[0]
        print(f"Spearman's rho en el conjunto de test: {rho:.4f}")

        if rho > rho_threshold:
            best_params = gs.best_params_
            break

    return best_params, rho

best_params, rho = find_best_model(X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, net, params, myCViterator, coef_spearman_NO_CNN)

print("best hyperparameter found:")
print(best_params)
print(f"Best Spearman's rho: {rho:.4f}")

In [ ]:
# save the variables necessary to generate Figure 1b and evaluate the results of the model.

In [ ]:
# Save the ensemble model (commented for security reasons)
# torch.save(gs.best_estimator_.module_.state_dict(), '../../Data/Calibrated_models/ensemble_model_V0.pth')

In [ ]:
np.save('../../Data/X_train_CNN_Bay_others.npy', X_train_CNN_Bay_others)

In [ ]:
torch.save(X_test_tensor, '../../Data/X_test_tensor.pt')

In [ ]:
torch.save(y_test_tensor, '../../Data/y_test_tensor.pt')